In [2]:
import numpy as np

In [3]:
N = 20
D = 5
Th = np.random.random((N, D))*2
x = np.random.random(D)

In [4]:
Th.dot(x)

array([ 3.00313859,  4.07645365,  1.58340697,  2.3587485 ,  4.23725839,
        2.71731624,  3.96497897,  3.38484423,  3.20907132,  2.84177447,
        3.63115584,  2.94250268,  4.05873768,  2.44655411,  3.24463398,
        3.29183882,  3.73503244,  1.30001887,  2.57425252,  3.47264893])

In [5]:
L = 4
thresholds = [1, 3, 5]

In [6]:
from math import exp

def sigmoid(x):
    return 1 / (1 + exp(-x))

sigma = np.vectorize(sigmoid)

In [7]:
cdf = sigma(np.outer(Th.dot(x), np.ones(3)) - thresholds)
#cdf

In [8]:
pdf = np.column_stack((np.ones(N), cdf))
for i in range(3):
    pdf[:, i] -= pdf[:, i + 1]
pdf

array([[ 0.11887378,  0.38034157,  0.3812518 ,  0.11953285],
       [ 0.04408904,  0.21008868,  0.46158643,  0.28423585],
       [ 0.35814903,  0.44665473,  0.16341535,  0.0317809 ],
       [ 0.20444378,  0.45059253,  0.27843342,  0.06653027],
       [ 0.03778745,  0.18712612,  0.45703511,  0.31805133],
       [ 0.15221717,  0.41798689,  0.33722867,  0.09256727],
       [ 0.04903332,  0.22684911,  0.46200574,  0.26211183],
       [ 0.08433573,  0.32062334,  0.42916691,  0.16587403],
       [ 0.09893883,  0.3489829 ,  0.40911937,  0.1429589 ],
       [ 0.13684156,  0.4026325 ,  0.35696086,  0.10356508],
       [ 0.06716   ,  0.2800885 ,  0.44994485,  0.20280665],
       [ 0.12537317,  0.3889972 ,  0.37233262,  0.11329701],
       [ 0.04484174,  0.21270902,  0.46180381,  0.28064543],
       [ 0.19053246,  0.44440224,  0.29286998,  0.07219533],
       [ 0.09581333,  0.34333136,  0.41348364,  0.14737166],
       [ 0.09180113,  0.33575263,  0.41904388,  0.15340237],
       [ 0.06093755,  0.

In [9]:
np.argmax(pdf, axis=1)

array([2, 2, 1, 1, 2, 1, 2, 2, 2, 1, 2, 1, 2, 1, 2, 2, 2, 0, 1, 2])

In [10]:
def logloss(x):
    return np.log(1 + np.exp(-x))

In [11]:
labels = np.round(np.random.random(N)*3)

In [12]:
labels

array([ 2.,  2.,  1.,  1.,  1.,  2.,  1.,  3.,  3.,  3.,  3.,  2.,  1.,
        1.,  2.,  2.,  2.,  2.,  1.,  0.])

In [13]:
sgn = np.fromfunction(lambda i, j: 2 * (1 + j < labels[i]) - 1, (N, L - 1), dtype=int)

In [14]:
delta = np.outer(Th.dot(x), np.ones(3)) - thresholds

In [15]:
logloss(sgn * delta).sum(axis=1)

array([ 0.94857476,  1.74921665,  1.27625979,  2.07937362,  5.15061829,
        0.8240276 ,  4.60699871,  0.78860141,  0.85251699,  1.0318727 ,
        0.7227384 ,  0.91901486,  4.79055526,  2.18709989,  1.0830773 ,
        1.11249691,  1.43821824,  0.74655946,  2.35000669,  3.70725926])

In [49]:
import tensorflow as tf

aa = tf.constant([1., 5., 3.])
a = tf.reshape(aa, (3, 1))
b = tf.ones([1, 3])

In [50]:
#sess = tf.InteractiveSession()
with tf.Session() as sess:
    print(sess.run(tf.matmul(a, b)))

[[ 1.  1.  1.]
 [ 5.  5.  5.]
 [ 3.  3.  3.]]


In [112]:
some_logits = Th.dot(x)
#thr = tf.cast(tf.constant(thresholds), tf.float64)
#lab = tf.constant(labels)

In [115]:
some_logits[:, None] - thresholds

array([[  2.00313859e+00,   3.13858669e-03,  -1.99686141e+00],
       [  3.07645365e+00,   1.07645365e+00,  -9.23546352e-01],
       [  5.83406973e-01,  -1.41659303e+00,  -3.41659303e+00],
       [  1.35874850e+00,  -6.41251495e-01,  -2.64125150e+00],
       [  3.23725839e+00,   1.23725839e+00,  -7.62741614e-01],
       [  1.71731624e+00,  -2.82683757e-01,  -2.28268376e+00],
       [  2.96497897e+00,   9.64978974e-01,  -1.03502103e+00],
       [  2.38484423e+00,   3.84844226e-01,  -1.61515577e+00],
       [  2.20907132e+00,   2.09071322e-01,  -1.79092868e+00],
       [  1.84177447e+00,  -1.58225534e-01,  -2.15822553e+00],
       [  2.63115584e+00,   6.31155842e-01,  -1.36884416e+00],
       [  1.94250268e+00,  -5.74973236e-02,  -2.05749732e+00],
       [  3.05873768e+00,   1.05873768e+00,  -9.41262321e-01],
       [  1.44655411e+00,  -5.53445890e-01,  -2.55344589e+00],
       [  2.24463398e+00,   2.44633978e-01,  -1.75536602e+00],
       [  2.29183882e+00,   2.91838818e-01,  -1.7081611

In [119]:
range(4) - labels[:, None]

array([[-2., -1.,  0.,  1.],
       [-2., -1.,  0.,  1.],
       [-1.,  0.,  1.,  2.],
       [-1.,  0.,  1.,  2.],
       [-1.,  0.,  1.,  2.],
       [-2., -1.,  0.,  1.],
       [-1.,  0.,  1.,  2.],
       [-3., -2., -1.,  0.],
       [-3., -2., -1.,  0.],
       [-3., -2., -1.,  0.],
       [-3., -2., -1.,  0.],
       [-2., -1.,  0.,  1.],
       [-1.,  0.,  1.,  2.],
       [-1.,  0.,  1.,  2.],
       [-2., -1.,  0.,  1.],
       [-2., -1.,  0.,  1.],
       [-2., -1.,  0.,  1.],
       [-2., -1.,  0.,  1.],
       [-1.,  0.,  1.,  2.],
       [ 0.,  1.,  2.,  3.]])

In [141]:
logits = tf.placeholder(tf.float32, shape=[None])
thr = tf.placeholder(tf.float32, shape=[None])
lab = tf.placeholder(tf.float32, shape=[None])

In [137]:
NB_CLASSES = 4

In [138]:
def tf_logloss(x):
    return tf.log(1 + tf.exp(-x))

In [142]:
def all_thresholds(labels, logits, thresholds):
    #batch_size = tf.size(labels).eval()
    #sgn = np.fromfunction(lambda i, j: 2 * (1 + j < labels.eval()[i]) - 1, (batch_size, NB_CLASSES - 1), dtype=int)
    signs = tf.sign(2 * (tf.range(NB_CLASSES, dtype=tf.float32) - labels[:, None]) + 1)
    delta = logits[:, None] - thresholds
    return tf.reduce_sum(tf_logloss(signs * delta), axis=1)

In [143]:
metrics = all_thresholds(lab, logits, thr)

In [127]:
np.random.random((5))[None, :].shape

(1, 5)

In [132]:
delta = logits[:, None] - thr

In [133]:
with tf.Session() as sess:
    print(sess.run(all_, feed_dict={logits: some_logits, thr: thresholds}))

[[  2.00313854e+00   3.13854218e-03  -1.99686146e+00]
 [  3.07645369e+00   1.07645369e+00  -9.23546314e-01]
 [  5.83406925e-01  -1.41659307e+00  -3.41659307e+00]
 [  1.35874844e+00  -6.41251564e-01  -2.64125156e+00]
 [  3.23725843e+00   1.23725843e+00  -7.62741566e-01]
 [  1.71731615e+00  -2.82683849e-01  -2.28268385e+00]
 [  2.96497893e+00   9.64978933e-01  -1.03502107e+00]
 [  2.38484430e+00   3.84844303e-01  -1.61515570e+00]
 [  2.20907140e+00   2.09071398e-01  -1.79092860e+00]
 [  1.84177446e+00  -1.58225536e-01  -2.15822554e+00]
 [  2.63115573e+00   6.31155729e-01  -1.36884427e+00]
 [  1.94250274e+00  -5.74972630e-02  -2.05749726e+00]
 [  3.05873775e+00   1.05873775e+00  -9.41262245e-01]
 [  1.44655418e+00  -5.53445816e-01  -2.55344582e+00]
 [  2.24463391e+00   2.44633913e-01  -1.75536609e+00]
 [  2.29183888e+00   2.91838884e-01  -1.70816112e+00]
 [  2.73503232e+00   7.35032320e-01  -1.26496768e+00]
 [  3.00018907e-01  -1.69998109e+00  -3.69998121e+00]
 [  1.57425261e+00  -4.25747

In [39]:
result = all_thresholds(logits, thr, lab)

ValueError: Cannot evaluate tensor using `eval()`: No default session is registered. Use `with sess.as_default()` or pass an explicit session to `eval(session=sess)`

In [46]:
with tf.Session() as sess:
    print(sess.run(all_thresholds(lab, logits, thr), feed_dict={lab: labels, logits: Th.dot(x), thr: thresholds}))

[ 1.6303095   1.20975196  1.60408551  1.75802107  1.24029919  1.88728056
  0.76261534  2.5268018   1.38458055  2.42626421  3.34016564  1.92546685
  0.98619173  4.00346476  2.00474428  1.57902     2.66738516  1.99304748
  3.28847223  1.96645723]
